# Model Training

## Import Library

In [1]:
import tensorflow as tf
import numpy as np 
import matplotlib.pyplot as plt
import os

2024-11-23 17:55:34.791329: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732359335.546224   37553 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732359335.764998   37553 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-23 17:55:37.825883: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Try one

The data has been processed manually to check if there are something different in it, there is some that might disturb the training to recognize it such as photo with people and the food in it, but the data from that might be usefull because the food is still at the center of the picture and this happened with a few of the food probably in total there are about 10 to 15~ picture of that occuring. It wont affect much as the majority of the data is clean so adding that to become a variety might be a good idea.
In the first test i would like to test if not augmenting the data much at first would effect the accuracy that it would plateau at. And the result is as such

In [2]:
train_dataset_temp = tf.keras.preprocessing.image_dataset_from_directory(
    './Datajadi/train',
    image_size=(256, 256),
    batch_size=16,
    shuffle=True,
    label_mode='categorical'
)

valid_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    './Datajadi/val',
    image_size=(256, 256),
    batch_size=16,
    label_mode='categorical'
)

test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    './Datajadi/test',
    image_size=(256, 256),
    batch_size=16,
    label_mode='categorical'
)

labelnya = train_dataset_temp.class_names
jumlah_label = len(labelnya)

Found 80800 files belonging to 101 classes.


I0000 00:00:1732216234.441880     660 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5563 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060, pci bus id: 0000:08:00.0, compute capability: 8.9


Found 10100 files belonging to 101 classes.
Found 10100 files belonging to 101 classes.


In [21]:
data_augment = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
    # tf.keras.layers.RandomHeight(0.1),
    # tf.keras.layers.RandomWidth(0.1),
    tf.keras.layers.RandomBrightness(0.2),
    tf.keras.layers.RandomContrast(0.2)
])
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset_temp.map(lambda x,y: (data_augment(x, training=True),y), num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.prefetch(AUTOTUNE)
valid_dataset = valid_dataset.prefetch(AUTOTUNE)

In [22]:
with open("label_listered.txt",'w') as file:
    for i in labelnya:
        file.write(f"{i}\n")

In [ ]:
print(len(train_dataset))

In [8]:
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="Checkpoint/Improving/Test_{epoch:02d}.keras",
    save_best_only=True,
    mode = max,
    monitor='val_accuracy',
    verbose=1
)

/tmp/ipykernel_459/1512639513.py:1: UserWarning: ModelCheckpoint mode '<built-in function max>' is unknown, fallback to auto mode.
  checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(


In [3]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-8,
    verbose=1
)

In [6]:
Efficientnet = tf.keras.applications.EfficientNetV2S(
include_top=False,
weights='imagenet',
input_shape = (256,256,3)
)
for layer in Efficientnet.layers:
    layer.trainable= False
last_layer = Efficientnet.get_layer('block6o_add')
last_output = last_layer.output
x = tf.keras.layers.BatchNormalization()(last_output)
x = tf.keras.layers.Conv2D(256, (3,3),activation ='relu')(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(512, activation='relu',kernel_regularizer=tf.keras.regularizers.L2(0.001))(x)
x = tf.keras.layers.Dropout(0.5)(x)
x_out = tf.keras.layers.Dense(jumlah_label, activation='sigmoid')(x)
model = tf.keras.Model(Efficientnet.input, x_out)
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [7]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 256, 256,  │          0 │ input_layer_1[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 128, 128,  │        648 │ rescaling[0][0]   │
│                     │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 128, 128,  │         96 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 128, 128,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 128, 128,  │      5,184 │ stem_activation[… │
│ (Conv2D)            │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_bn  │ (None, 128, 128,  │         96 │ block1a_project_… │
│ (BatchNormalizatio… │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_ac… │ (None, 128, 128,  │          0 │ block1a_project_… │
│ (Activation)        │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_add (Add)   │ (None, 128, 128,  │          0 │ block1a_project_… │
│                     │ 24)               │            │ stem_activation[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_project_co… │ (None, 128, 128,  │      5,184 │ block1a_add[0][0] │
│ (Conv2D)            │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_project_bn  │ (None, 128, 128,  │         96 │ block1b_project_… │
│ (BatchNormalizatio… │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_project_ac… │ (None, 128, 128,  │          0 │ block1b_project_… │
│ (Activation)        │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_drop        │ (None, 128, 128,  │          0 │ block1b_project_… │
│ (Dropout)           │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_add (Add)   │ (None, 128, 128,  │          0 │ block1b_drop[0][… │
│                     │ 24)               │            │ block1a_add[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_expand_conv │ (None, 64, 64,    │     20,736 │ block1b_add[0][0] │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_expand_bn   │ (None, 64, 64,    │        384 │ block2a_expand_c… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_expand_act… │ (None, 64, 64,    │          0 │ block2a_expand_b

 Total params: 23,464,133 (89.51 MB)

 Trainable params: 3,132,773 (11.95 MB)

 Non-trainable params: 20,331,360 (77.56 MB)

In [9]:
model.fit(train_dataset, epochs=10, validation_data=valid_dataset, callbacks=[checkpoint_callback, reduce_lr])

Epoch 1/10


I0000 00:00:1732216291.852282     905 service.cc:148] XLA service 0x7feff00020a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1732216291.882762     905 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 4060, Compute Capability 8.9
2024-11-22 02:11:32.874281: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1732216294.762656     905 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1732216315.351358     905 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


5050/5050 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.2301 - loss: 0.0755

2024-11-22 02:15:26.021870: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_4257', 8 bytes spill stores, 8 bytes spill loads




Epoch 1: saving model to /Checkpoint/cattlepoch_01.weights.h5
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 253s 43ms/step - accuracy: 0.2301 - loss: 0.0755 - val_accuracy: 0.5002 - val_loss: 0.0320 - learning_rate: 0.0010
Epoch 2/10
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step - accuracy: 0.4340 - loss: 0.0368

...


Epoch 8: saving model to /Checkpoint/cattlepoch_08.weights.h5
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 586s 116ms/step - accuracy: 0.5483 - loss: 0.0326 - val_accuracy: 0.6340 - val_loss: 0.0271 - learning_rate: 0.0010
Epoch 9/10
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step - accuracy: 0.5562 - loss: 0.0322

2024-11-22 03:34:10.729502: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 16783872 bytes after encountering the first element of size 16783872 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size
2024-11-22 03:34:10.729644: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 16783872 bytes after encountering the first element of size 16783872 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size



Epoch 9: saving model to /Checkpoint/cattlepoch_09.weights.h5
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 586s 116ms/step - accuracy: 0.5562 - loss: 0.0322 - val_accuracy: 0.6421 - val_loss: 0.0270 - learning_rate: 0.0010
Epoch 10/10
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step - accuracy: 0.5588 - loss: 0.0322

2024-11-22 03:43:55.460769: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 16783872 bytes after encountering the first element of size 16783872 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size
2024-11-22 03:43:55.460879: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 16783872 bytes after encountering the first element of size 16783872 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size



Epoch 10: saving model to /Checkpoint/cattlepoch_10.weights.h5
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 582s 115ms/step - accuracy: 0.5588 - loss: 0.0322 - val_accuracy: 0.6424 - val_loss: 0.0267 - learning_rate: 0.0010


In [10]:
model.save('TranferRevised2/Categorical_TlEfnet_10_epochs.h5')

In [11]:
mulaidari = "block6a_expand_conv"
Kondisi = False
for layer in model.layers:
    if layer.name == mulaidari:
        Kondisi = True
    layer.trainable = Kondisi
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 256, 256,  │          0 │ input_layer_1[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 128, 128,  │        648 │ rescaling[0][0]   │
│                     │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 128, 128,  │         96 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 128, 128,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 128, 128,  │      5,184 │ stem_activation[… │
│ (Conv2D)            │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_bn  │ (None, 128, 128,  │         96 │ block1a_project_… │
│ (BatchNormalizatio… │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_ac… │ (None, 128, 128,  │          0 │ block1a_project_… │
│ (Activation)        │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_add (Add)   │ (None, 128, 128,  │          0 │ block1a_project_… │
│                     │ 24)               │            │ stem_activation[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_project_co… │ (None, 128, 128,  │      5,184 │ block1a_add[0][0] │
│ (Conv2D)            │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_project_bn  │ (None, 128, 128,  │         96 │ block1b_project_… │
│ (BatchNormalizatio… │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_project_ac… │ (None, 128, 128,  │          0 │ block1b_project_… │
│ (Activation)        │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_drop        │ (None, 128, 128,  │          0 │ block1b_project_… │
│ (Dropout)           │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_add (Add)   │ (None, 128, 128,  │          0 │ block1b_drop[0][… │
│                     │ 24)               │            │ block1a_add[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_expand_conv │ (None, 64, 64,    │     20,736 │ block1b_add[0][0] │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_expand_bn   │ (None, 64, 64,    │        384 │ block2a_expand_c… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_expand_act… │ (None, 64, 64,    │          0 │ block2a_expand_b

 Total params: 22,321,041 (85.15 MB)

 Trainable params: 15,335,821 (58.50 MB)

 Non-trainable params: 5,437,240 (20.74 MB)

 Optimizer params: 1,547,980 (5.91 MB)

In [12]:
model.fit(train_dataset, epochs=20, validation_data=valid_dataset, callbacks=[checkpoint_callback, reduce_lr],initial_epoch=10)

Epoch 11/20
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step - accuracy: 0.5624 - loss: 0.0318

2024-11-22 03:53:40.722625: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 16783872 bytes after encountering the first element of size 16783872 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size



Epoch 11: saving model to /Checkpoint/cattlepoch_11.weights.h5
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 584s 115ms/step - accuracy: 0.5624 - loss: 0.0318 - val_accuracy: 0.6562 - val_loss: 0.0260 - learning_rate: 0.0010
Epoch 12/20
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step - accuracy: 0.5646 - loss: 0.0317

2024-11-22 04:03:22.046455: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 16783872 bytes after encountering the first element of size 16783872 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size
2024-11-22 04:03:22.046524: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 16783872 bytes after encountering the first element of size 16783872 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size



Epoch 12: saving model to /Checkpoint/cattlepoch_12.weights.h5
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 581s 115ms/step - accuracy: 0.5646 - loss: 0.0317 - val_accuracy: 0.6510 - val_loss: 0.0263 - learning_rate: 0.0010
Epoch 13/20
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.5686 - loss: 0.0317

...


Epoch 18: saving model to /Checkpoint/cattlepoch_18.weights.h5

Epoch 18: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 578s 114ms/step - accuracy: 0.6013 - loss: 0.0270 - val_accuracy: 0.6792 - val_loss: 0.0222 - learning_rate: 5.0000e-04
Epoch 19/20
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step - accuracy: 0.6140 - loss: 0.0251

2024-11-22 05:06:46.605556: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 16783872 bytes after encountering the first element of size 16783872 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size
2024-11-22 05:06:46.605692: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 16783872 bytes after encountering the first element of size 16783872 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size



Epoch 19: saving model to /Checkpoint/cattlepoch_19.weights.h5
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 576s 114ms/step - accuracy: 0.6140 - loss: 0.0251 - val_accuracy: 0.6890 - val_loss: 0.0199 - learning_rate: 2.5000e-04
Epoch 20/20
   5/5050 ━━━━━━━━━━━━━━━━━━━━ 2:54 35ms/step - accuracy: 0.6938 - loss: 0.0209 

2024-11-22 05:07:03.701251: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 12589568 bytes after encountering the first element of size 12589568 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


5050/5050 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.6179 - loss: 0.0244
Epoch 20: saving model to /Checkpoint/cattlepoch_20.weights.h5
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 247s 49ms/step - accuracy: 0.6179 - loss: 0.0244 - val_accuracy: 0.6913 - val_loss: 0.0198 - learning_rate: 2.5000e-04


In [13]:
model.save('TranferRevised2/Categorical_TlEfnet_20_epochs.h5')

In [14]:
val_loss, val_acc = model.evaluate(valid_dataset)
tra_loss, tra_acc = model.evaluate(train_dataset)
test_loss, test_acc = model.evaluate(test_dataset)


print(f"Train Accuracy\t\t: {tra_acc * 100:.2f}% | Train Loss\t\t: {tra_loss}")
print(f"Validation Accuracy\t: {val_acc * 100:.2f}% | Validation Loss\t: {val_loss}")
print(f"Test Accuracy\t\t: {test_acc * 100:.2f}% | Test Loss \t\t: {test_loss}")

632/632 ━━━━━━━━━━━━━━━━━━━━ 16s 26ms/step - accuracy: 0.6827 - loss: 0.0200
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 559s 111ms/step - accuracy: 0.6712 - loss: 0.0209
632/632 ━━━━━━━━━━━━━━━━━━━━ 17s 26ms/step - accuracy: 0.6994 - loss: 0.0195
Train Accuracy		: 67.25% | Train Loss		: 0.020682862028479576
Validation Accuracy	: 69.13% | Validation Loss	: 0.019754208624362946
Test Accuracy		: 68.79% | Test Loss 		: 0.019830694422125816


In [15]:
mulaidari = "block5a_expand_conv"
Kondisi = False
for layer in model.layers:
    if layer.name == mulaidari:
        Kondisi = True
    layer.trainable = Kondisi
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 256, 256,  │          0 │ input_layer_1[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 128, 128,  │        648 │ rescaling[0][0]   │
│                     │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 128, 128,  │         96 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 128, 128,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 128, 128,  │      5,184 │ stem_activation[… │
│ (Conv2D)            │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_bn  │ (None, 128, 128,  │         96 │ block1a_project_… │
│ (BatchNormalizatio… │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_ac… │ (None, 128, 128,  │          0 │ block1a_project_… │
│ (Activation)        │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_add (Add)   │ (None, 128, 128,  │          0 │ block1a_project_… │
│                     │ 24)               │            │ stem_activation[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_project_co… │ (None, 128, 128,  │      5,184 │ block1a_add[0][0] │
│ (Conv2D)            │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_project_bn  │ (None, 128, 128,  │         96 │ block1b_project_… │
│ (BatchNormalizatio… │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_project_ac… │ (None, 128, 128,  │          0 │ block1b_project_… │
│ (Activation)        │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_drop        │ (None, 128, 128,  │          0 │ block1b_project_… │
│ (Dropout)           │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_add (Add)   │ (None, 128, 128,  │          0 │ block1b_drop[0][… │
│                     │ 24)               │            │ block1a_add[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_expand_conv │ (None, 64, 64,    │     20,736 │ block1b_add[0][0] │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_expand_bn   │ (None, 64, 64,    │        384 │ block2a_expand_c… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_expand_act… │ (None, 64, 64,    │          0 │ block2a_expand_b

 Total params: 22,321,041 (85.15 MB)

 Trainable params: 18,799,661 (71.72 MB)

 Non-trainable params: 1,973,400 (7.53 MB)

 Optimizer params: 1,547,980 (5.91 MB)

In [16]:
model.fit(train_dataset, epochs=40, validation_data=valid_dataset, callbacks=[checkpoint_callback, reduce_lr],initial_epoch=20)

Epoch 21/40
   5/5050 ━━━━━━━━━━━━━━━━━━━━ 2:57 35ms/step - accuracy: 0.6248 - loss: 0.0249 

2024-11-22 05:21:04.257500: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 12589568 bytes after encountering the first element of size 12589568 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


5050/5050 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.6209 - loss: 0.0243
Epoch 21: saving model to /Checkpoint/cattlepoch_21.weights.h5
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 207s 41ms/step - accuracy: 0.6209 - loss: 0.0243 - val_accuracy: 0.6908 - val_loss: 0.0198 - learning_rate: 2.5000e-04
Epoch 22/40
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.6193 - loss: 0.0242

2024-11-22 05:27:37.772362: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 16783872 bytes after encountering the first element of size 16783872 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size
2024-11-22 05:27:37.772468: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 16783872 bytes after encountering the first element of size 16783872 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size



Epoch 22: saving model to /Checkpoint/cattlepoch_22.weights.h5
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 204s 40ms/step - accuracy: 0.6193 - loss: 0.0242 - val_accuracy: 0.6903 - val_loss: 0.0198 - learning_rate: 2.5000e-04
Epoch 23/40
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step - accuracy: 0.6212 - loss: 0.0241

...


Epoch 38: saving model to /Checkpoint/cattlepoch_38.weights.h5
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 253s 50ms/step - accuracy: 0.6472 - loss: 0.0208 - val_accuracy: 0.7037 - val_loss: 0.0171 - learning_rate: 3.1250e-05
Epoch 39/40
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step - accuracy: 0.6455 - loss: 0.0208

2024-11-22 08:05:36.077150: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 16783872 bytes after encountering the first element of size 16783872 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size
2024-11-22 08:05:36.077240: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 16783872 bytes after encountering the first element of size 16783872 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size



Epoch 39: saving model to /Checkpoint/cattlepoch_39.weights.h5
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 583s 115ms/step - accuracy: 0.6455 - loss: 0.0208 - val_accuracy: 0.7017 - val_loss: 0.0171 - learning_rate: 3.1250e-05
Epoch 40/40
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step - accuracy: 0.6464 - loss: 0.0208

2024-11-22 08:15:14.795517: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 16783872 bytes after encountering the first element of size 16783872 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size
2024-11-22 08:15:14.795608: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 16783872 bytes after encountering the first element of size 16783872 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size



Epoch 40: saving model to /Checkpoint/cattlepoch_40.weights.h5
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 580s 115ms/step - accuracy: 0.6464 - loss: 0.0208 - val_accuracy: 0.7045 - val_loss: 0.0171 - learning_rate: 3.1250e-05


In [17]:
model.save('TranferRevised2/Categorical_TlEfnet_40_epochs.h5')

In [18]:
val_loss, val_acc = model.evaluate(valid_dataset)
tra_loss, tra_acc = model.evaluate(train_dataset)
test_loss, test_acc = model.evaluate(test_dataset)


print(f"Train Accuracy\t\t: {tra_acc * 100:.2f}% | Train Loss\t\t: {tra_loss}")
print(f"Validation Accuracy\t: {val_acc * 100:.2f}% | Validation Loss\t: {val_loss}")
print(f"Test Accuracy\t\t: {test_acc * 100:.2f}% | Test Loss \t\t: {test_loss}")

  7/632 ━━━━━━━━━━━━━━━━━━━━ 16s 26ms/step - accuracy: 0.6610 - loss: 0.0201

2024-11-22 08:15:32.996015: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 16783872 bytes after encountering the first element of size 16783872 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size
2024-11-22 08:15:32.996155: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 16783872 bytes after encountering the first element of size 16783872 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


632/632 ━━━━━━━━━━━━━━━━━━━━ 16s 25ms/step - accuracy: 0.7015 - loss: 0.0173
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 317s 63ms/step - accuracy: 0.6927 - loss: 0.0178
632/632 ━━━━━━━━━━━━━━━━━━━━ 16s 25ms/step - accuracy: 0.7135 - loss: 0.0169
Train Accuracy		: 69.46% | Train Loss		: 0.017641032114624977
Validation Accuracy	: 70.45% | Validation Loss	: 0.01710057258605957
Test Accuracy		: 70.45% | Test Loss 		: 0.017190372571349144


In [19]:
mulaidari = "block3a_expand_conv"
Kondisi = False
for layer in model.layers:
    if layer.name == mulaidari:
        Kondisi = True
    layer.trainable = Kondisi
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 256, 256,  │          0 │ input_layer_1[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 128, 128,  │        648 │ rescaling[0][0]   │
│                     │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 128, 128,  │         96 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 128, 128,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 128, 128,  │      5,184 │ stem_activation[… │
│ (Conv2D)            │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_bn  │ (None, 128, 128,  │         96 │ block1a_project_… │
│ (BatchNormalizatio… │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_ac… │ (None, 128, 128,  │          0 │ block1a_project_… │
│ (Activation)        │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_add (Add)   │ (None, 128, 128,  │          0 │ block1a_project_… │
│                     │ 24)               │            │ stem_activation[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_project_co… │ (None, 128, 128,  │      5,184 │ block1a_add[0][0] │
│ (Conv2D)            │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_project_bn  │ (None, 128, 128,  │         96 │ block1b_project_… │
│ (BatchNormalizatio… │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_project_ac… │ (None, 128, 128,  │          0 │ block1b_project_… │
│ (Activation)        │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_drop        │ (None, 128, 128,  │          0 │ block1b_project_… │
│ (Dropout)           │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_add (Add)   │ (None, 128, 128,  │          0 │ block1b_drop[0][… │
│                     │ 24)               │            │ block1a_add[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_expand_conv │ (None, 64, 64,    │     20,736 │ block1b_add[0][0] │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_expand_bn   │ (None, 64, 64,    │        384 │ block2a_expand_c… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_expand_act… │ (None, 64, 64,    │          0 │ block2a_expand_b

 Total params: 22,321,041 (85.15 MB)

 Trainable params: 20,306,525 (77.46 MB)

 Non-trainable params: 466,536 (1.78 MB)

 Optimizer params: 1,547,980 (5.91 MB)

In [20]:
model.fit(train_dataset, epochs=60, validation_data=valid_dataset, callbacks=[checkpoint_callback, reduce_lr],initial_epoch=40)

Epoch 41/60
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step - accuracy: 0.6460 - loss: 0.0208

2024-11-22 08:30:44.539173: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 16783872 bytes after encountering the first element of size 16783872 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size
2024-11-22 08:30:44.539272: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 16783872 bytes after encountering the first element of size 16783872 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size



Epoch 41: saving model to /Checkpoint/cattlepoch_41.weights.h5
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 580s 115ms/step - accuracy: 0.6460 - loss: 0.0208 - val_accuracy: 0.7020 - val_loss: 0.0171 - learning_rate: 3.1250e-05
Epoch 42/60
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step - accuracy: 0.6468 - loss: 0.0207

2024-11-22 08:40:21.424856: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 16783872 bytes after encountering the first element of size 16783872 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size
2024-11-22 08:40:21.424935: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 16783872 bytes after encountering the first element of size 16783872 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size



Epoch 42: saving model to /Checkpoint/cattlepoch_42.weights.h5
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 576s 114ms/step - accuracy: 0.6468 - loss: 0.0207 - val_accuracy: 0.7044 - val_loss: 0.0171 - learning_rate: 3.1250e-05
Epoch 43/60
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step - accuracy: 0.6460 - loss: 0.0208

...

2024-11-22 10:07:00.144618: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 16783872 bytes after encountering the first element of size 16783872 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size
2024-11-22 10:07:00.144715: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 16783872 bytes after encountering the first element of size 16783872 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size



Epoch 51: saving model to /Checkpoint/cattlepoch_51.weights.h5

Epoch 51: ReduceLROnPlateau reducing learning rate to 7.812500371073838e-06.
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 578s 114ms/step - accuracy: 0.6499 - loss: 0.0203 - val_accuracy: 0.7059 - val_loss: 0.0168 - learning_rate: 1.5625e-05
Epoch 52/60
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step - accuracy: 0.6503 - loss: 0.0203

2024-11-22 10:16:37.082140: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 16783872 bytes after encountering the first element of size 16783872 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size
2024-11-22 10:16:37.082233: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 16783872 bytes after encountering the first element of size 16783872 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size



Epoch 52: saving model to /Checkpoint/cattlepoch_52.weights.h5
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 578s 114ms/step - accuracy: 0.6503 - loss: 0.0203 - val_accuracy: 0.7048 - val_loss: 0.0168 - learning_rate: 7.8125e-06
Epoch 53/60
1145/5050 ━━━━━━━━━━━━━━━━━━━━ 7:14 111ms/step - accuracy: 0.6523 - loss: 0.0201

KeyboardInterrupt: 

In [22]:
model.save('TranferRevised2/Categorical_TlEfnet_52_epochs.h5')

In [23]:
val_loss, val_acc = model.evaluate(valid_dataset)
tra_loss, tra_acc = model.evaluate(train_dataset)
test_loss, test_acc = model.evaluate(test_dataset)


print(f"Train Accuracy\t\t: {tra_acc * 100:.2f}% | Train Loss\t\t: {tra_loss}")
print(f"Validation Accuracy\t: {val_acc * 100:.2f}% | Validation Loss\t: {val_loss}")
print(f"Test Accuracy\t\t: {test_acc * 100:.2f}% | Test Loss \t\t: {test_loss}")

2024-11-22 10:20:27.512784: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 16783872 bytes after encountering the first element of size 16783872 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size
2024-11-22 10:20:27.512884: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 16783872 bytes after encountering the first element of size 16783872 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


632/632 ━━━━━━━━━━━━━━━━━━━━ 17s 27ms/step - accuracy: 0.6974 - loss: 0.0170
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 556s 110ms/step - accuracy: 0.7018 - loss: 0.0169
632/632 ━━━━━━━━━━━━━━━━━━━━ 16s 25ms/step - accuracy: 0.7182 - loss: 0.0165
Train Accuracy		: 70.22% | Train Loss		: 0.016956036910414696
Validation Accuracy	: 70.48% | Validation Loss	: 0.016755882650613785
Test Accuracy		: 70.84% | Test Loss 		: 0.016868172213435173


i stopped it there becuase any further and it seems to be plateauing. It seems like with minimum data augmentation, the highest accuracy it would reach wouldnt be high and would reach somewhere in 70%. After this i would train it further not from the beginning but change the dataset with more augmenting using imagegenerator. This mean that i would have to load the data differently than before but the label should at least be the same. My logic behind this is because the model has learn the feature with minimum augmentation flipping, random brightness, contrast etc, but if i change it a little bit more with cropping, shearing, it could learn more about the food that is in the image.

#### Training further

In [4]:
model = tf.keras.models.load_model('TranferRevised2/Categorical_TlEfnet_52_epochs.h5')

I0000 00:00:1732282933.266218     460 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5563 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060, pci bus id: 0000:08:00.0, compute capability: 8.9


In [5]:
traingen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range = 20,
    width_shift_range= 0.2,
    height_shift_range= 0.2,
    brightness_range= [0.9, 1.2],
    shear_range= 0.2,
    zoom_range= [0.95,1.2],
    horizontal_flip = True,
    fill_mode= 'nearest',
)

valgen = tf.keras.preprocessing.image.ImageDataGenerator(
)

train_dataset = traingen.flow_from_directory(
    './Datajadi/train',
    target_size=(256,256),
    class_mode= 'categorical',
    batch_size= 16,
    shuffle=True
)
val_dataset = valgen.flow_from_directory(
    './Datajadi/val',
    target_size=(256,256),
    class_mode= 'categorical',
    batch_size= 16,
)

Found 80800 images belonging to 101 classes.
Found 10100 images belonging to 101 classes.


In [6]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [9]:
model.fit(train_dataset, epochs=20, validation_data=val_dataset)

/home/usle/Bangkit/myenv/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step - accuracy: 0.4688 - loss: 0.0364

2024-11-22 21:03:10.853343: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_4113', 8 bytes spill stores, 8 bytes spill loads



5050/5050 ━━━━━━━━━━━━━━━━━━━━ 866s 171ms/step - accuracy: 0.4688 - loss: 0.0364 - val_accuracy: 0.6065 - val_loss: 0.0292
Epoch 2/20
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 897s 178ms/step - accuracy: 0.5936 - loss: 0.0299 - val_accuracy: 0.6708 - val_loss: 0.0248
Epoch 3/20
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 848s 168ms/step - accuracy: 0.6348 - loss: 0.0274 - val_accuracy: 0.6736 - val_loss: 0.0246
Epoch 4/20
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 867s 172ms/step - accuracy: 0.6668 - loss: 0.0254 - val_accuracy: 0.6974 - val_loss: 0.0238
Epoch 5/20
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 861s 170ms/step - accuracy: 0.6908 - loss: 0.0242 - val_accuracy: 0.7120 - val_loss: 0.0225
Epoch 6/20
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 834s 165ms/step - accuracy: 0.7096 - loss: 0.0230 - val_accuracy: 0.7206 - val_loss: 0.0223
Epoch 7/20
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 828s 164ms/step - accuracy: 0.7286 - loss: 0.0221 - val_accuracy: 0.7220 - val_loss: 0.0222
Epoch 8/20
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 815s 161ms/step - accuracy: 0.7

From the result above it seems my hunch was correct. so lets train it a bit further to see if it can generalize well and if there is a sign of overfitting.

In [12]:
model.save('TranferRevised2/77Percent.keras')

In [4]:
model = tf.keras.models.load_model('TranferRevised2/77Percent.keras')

I0000 00:00:1732302765.305262     459 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5563 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060, pci bus id: 0000:08:00.0, compute capability: 8.9


In [6]:
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    './Datajadi/test',
    image_size=(256, 256),
    batch_size=16,
    label_mode='categorical'
)

Found 10100 files belonging to 101 classes.


In [7]:
val_loss, val_acc = model.evaluate(val_dataset)
tra_loss, tra_acc = model.evaluate(train_dataset)
test_loss, test_acc = model.evaluate(test_dataset)


print(f"Train Accuracy\t\t: {tra_acc * 100:.2f}% | Train Loss\t\t: {tra_loss}")
print(f"Validation Accuracy\t: {val_acc * 100:.2f}% | Validation Loss\t: {val_loss}")
print(f"Test Accuracy\t\t: {test_acc * 100:.2f}% | Test Loss \t\t: {test_loss}")

/home/usle/Bangkit/myenv/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1732302995.380352     578 service.cc:148] XLA service 0x7ff628002060 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1732302995.380893     578 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 4060, Compute Capability 8.9
2024-11-23 02:16:35.603351: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1732302996.408626     578 cuda_dnn.cc:529] Loaded cuDNN version 90300


  3/632 ━━━━━━━━━━━━━━━━━━━━ 18s 29ms/step - accuracy: 0.7951 - loss: 0.0194   

I0000 00:00:1732303005.661187     578 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


631/632 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.7749 - loss: 0.0200

2024-11-23 02:17:15.087292: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_4113', 8 bytes spill stores, 8 bytes spill loads



632/632 ━━━━━━━━━━━━━━━━━━━━ 48s 53ms/step - accuracy: 0.7750 - loss: 0.0200
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 884s 175ms/step - accuracy: 0.8751 - loss: 0.0126
632/632 ━━━━━━━━━━━━━━━━━━━━ 18s 28ms/step - accuracy: 0.7810 - loss: 0.0206
Train Accuracy		: 87.58% | Train Loss		: 0.012534650042653084
Validation Accuracy	: 77.88% | Validation Loss	: 0.0199283417314291
Test Accuracy		: 77.96% | Test Loss 		: 0.019992945715785027


i made a mistake in cell below, my pc shutdown and didnt change the cell below, but the training didnt get affected and can still be continued

In [10]:
model.fit(train_dataset, epochs=40, validation_data=val_dataset, callbacks=[checkpoint_callback, reduce_lr],initial_epoch=20)

Epoch 21/40


2024-11-23 02:33:25.847622: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng57{k2=0,k13=2,k14=2,k18=1,k23=0} for conv (f32[256,64,1,1]{3,2,1,0}, u8[0]{0}) custom-call(f32[16,64,32,32]{3,2,1,0}, f32[16,256,32,32]{3,2,1,0}), window={size=1x1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardFilter", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kNone","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]} is taking a while...
2024-11-23 02:33:25.854784: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 2.947495587s
Trying algorithm eng57{k2=0,k13=2,k14=2,k18=1,k23=0} for conv (f32[256,64,1,1]{3,2,1,0}, u8[0]{0}) custom-call(f32[16,64,32,32]{3,2,1,0}, f32[16,256,32,32]{3,2,1,0}), window={size=1x1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardFilter", backend_config={"cu

5050/5050 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step - accuracy: 0.8318 - loss: 0.0158
Epoch 21: val_accuracy improved from -inf to 0.77119, saving model to Checkpoint/Improving/Test_21.keras
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 905s 168ms/step - accuracy: 0.8318 - loss: 0.0158 - val_accuracy: 0.7712 - val_loss: 0.0206 - learning_rate: 0.0010
Epoch 22/40
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step - accuracy: 0.8308 - loss: 0.0157
Epoch 22: val_accuracy did not improve from 0.77119
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 844s 167ms/step - accuracy: 0.8308 - loss: 0.0157 - val_accuracy: 0.7682 - val_loss: 0.0223 - learning_rate: 0.0010
Epoch 23/40
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step - accuracy: 0.8389 - loss: 0.0153
Epoch 23: val_accuracy did not improve from 0.77119
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 847s 168ms/step - accuracy: 0.8389 - loss: 0.0153 - val_accuracy: 0.7660 - val_loss: 0.0214 - learning_rate: 0.0010
Epoch 24/40
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step - accuracy: 0.8423 - loss: 0.015

In [15]:
val_loss, val_acc = model.evaluate(val_dataset)
tra_loss, tra_acc = model.evaluate(train_dataset)
test_loss, test_acc = model.evaluate(test_dataset)


print(f"Train Accuracy\t\t: {tra_acc * 100:.2f}% | Train Loss\t\t: {tra_loss}")
print(f"Validation Accuracy\t: {val_acc * 100:.2f}% | Validation Loss\t: {val_loss}")
print(f"Test Accuracy\t\t: {test_acc * 100:.2f}% | Test Loss \t\t: {test_loss}")

632/632 ━━━━━━━━━━━━━━━━━━━━ 15s 23ms/step - accuracy: 0.8168 - loss: 0.0165
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 748s 148ms/step - accuracy: 0.9691 - loss: 0.0026
632/632 ━━━━━━━━━━━━━━━━━━━━ 19s 31ms/step - accuracy: 0.8290 - loss: 0.0205
Train Accuracy		: 97.00% | Train Loss		: 0.002546892501413822
Validation Accuracy	: 82.04% | Validation Loss	: 0.01621641404926777
Test Accuracy		: 83.24% | Test Loss 		: 0.01697530224919319


In [14]:
model.save('TranferRevised2/Overfitted_Test.keras')

The model Overfitted but achived accuracy of 82% in validation and 83% in test. the difference is due to some data mismatch between validation and test data. but the best one that isnt overfitter is in Checkpoint/Improving/Test_25.keras where its 86 and 80 .